# 데이터수집
- 저작권정보 크롤링해오기

In [37]:
# 필요한 라이브러리 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# seaborn lib add
import seaborn as sns

from glob import glob

%matplotlib inline

import json

import warnings
warnings.filterwarnings(action = 'ignore') 


from datetime import date, datetime, timedelta
from dateutil.parser import parse

print("numpy version:",np.__version__) 
print("pandas version:",pd.__version__)

# 한글 폰트 문제 해결
import platform

from matplotlib import font_manager, rc

if platform.system() == 'Darwin':
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    plt.rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~') 


# 차트 축 <- 음수 부호 지원
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False

numpy version: 1.20.1
pandas version: 1.2.4


In [38]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, urlretrieve
from urllib.error   import HTTPError
from urllib.error   import URLError
import requests
import re
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

In [39]:
# chromedriver 변수 설정
path = 'C:/Users/whgud/driver/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get('https://www.musicow.com/market?tab=list&sortorder=amt_royalty_desc')

In [40]:
# 페이지를 지정해줄 url 변수설정
url = "https://www.musicow.com/market?tab=list&keyword=&sortorder=amt_royalty_desc&like=page=3&page="

In [41]:
# 값을 받아올 리스트들 생성

title_list = []
artist_list = []

저작권료_2017 = []
저작권료_2018 = []
저작권료_2019 = []
저작권료_2020 = []
최근1년저작권료_1주기준 = []
방송 = []
전송 = []
복제 = []
공연 = []
해외 = []
기타 = []

저작권공표일자 = []
작곡 = []
작사 = []
편곡 = []
저작권료지분=[]

In [23]:
# 크롤링 코드
for page in range(1,48):
    driver.get(url+str(page))
    time.sleep(1)
    
    for i in range(1,21):
        # 음원 1개 누르는 버튼
        music_btn = driver.find_element_by_xpath('//*[@id="market_list"]/div[2]/a['+str(i)+']')
        music_btn.send_keys(Keys.ENTER)
        time.sleep(1.5)
        
        # 곡정보 보여주는 버튼
        show_info_btn = driver.find_element_by_xpath('//*[@id="market_list"]/div[2]/div/div[5]/div[1]/a')
        show_info_btn.send_keys(Keys.ENTER)
        time.sleep(1)
        
        # 곡제목 크롤링후 리스트에 추가
        title = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[1]/div[2]/p/strong').text
        title_list.append(title)    
        
        # 아티스트명 크롤링후 리스트에 추가
        artist = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[1]/div[2]/em').text
        artist_list.append(artist)
        
        # 연도별 저작권료 크롤링
        five_year_btn =driver.find_element_by_xpath('//*[@id="song_info_royalty"]/div[2]/div/div/div[1]/div/div[1]/div/a[2]')
        five_year_btn.send_keys(Keys.ENTER)
        time.sleep(1)
        
        # 연도별 저작권료 각 연도에 맞게 리스트에 추가
        for year in range(2,6) : 
            fee = driver.find_element_by_xpath('//*[@id="graph1"]/span['+str(year)+']').get_attribute("data-bar-value")
            if year == 2 :
                저작권료_2017.append(fee)
            elif year == 3:
                저작권료_2018.append(fee)
            elif year == 4 : 
                저작권료_2019.append(fee)
            elif year == 5:
                저작권료_2020.append(fee)
        저작권료_2021 = driver.find_element_by_xpath('//*[@id="song_info_royalty"]/div[2]/div/div/div[2]/div/div[1]/strong').text
        최근1년저작권료_1주기준.append(저작권료_2021)
        
        # 매체별 저작권료 분포 크롤링후 각 매체에 맞게 리스트에 추가
        for idx in range(1,7):
            media = driver.find_element_by_xpath('//*[@id="song_info_royalty"]/div[2]/div/div/div[2]/div/div[2]/div[2]/dl['+str(idx)+']/dd').text
            if idx == 1:
                방송.append(media)
            if idx == 2:
                전송.append(media)
            if idx == 3:
                복제.append(media)
            if idx == 4:
                공연.append(media)
            if idx == 5:
                해외.append(media)
            if idx == 6:
                기타.append(media)
        
        # 저작권 공표일자 크롤링후 리스트에 추가
        date = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[2]/section[5]/div[2]/div/dl/dd[1]').text
        저작권공표일자.append(date)
        
        # 작곡가 크롤링후 리스트에 추가
        composition = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[2]/section[5]/div[2]/div/dl/dd[3]').text
        작곡.append(composition)
        
        # 작사가 크롤링후 리스트에 추가
        writer = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[2]/section[5]/div[2]/div/dl/dd[4]').text
        작사.append(writer)
        
        # 편곡자 크롤링후 리스트에 추가
        arrangement = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[2]/section[5]/div[2]/div/dl/dd[5]').text
        편곡.append(arrangement)
        
        
        # 저작권료 지분 각페이지마다 xpath가 달라서 for문으로 조건줘서 크롤링후 리스트에 추가
        for num in range(8,11):
            
            지분 = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[2]/section[5]/div[2]/div/dl/dt['+str(num)+']').text
            if 지분 == '저작권료 지분':
                stake = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[2]/section[5]/div[2]/div/dl/dd['+str(num)+']/p').text
                저작권료지분.append(stake)
                break # 없을시 오류발생

        
        time.sleep(1.5)
        
        # 디버깅 문장 출력
        print(title,"--",artist,"-- done" ) 
   
        # back_btn - 웹페이지 크기가 작을때만 활성화되는 버튼이라 사이즈 작게하고 크롤링해야함! 
        back_btn = driver.find_element_by_xpath('//*[@id="page_market"]/div[1]/div/a') 
        back_btn.send_keys(Keys.ENTER)

롤린 (Rollin') -- 브레이브걸스 -- done
축가 -- 전우성 (노을) -- done
오늘 그녀가 -- 루그 -- done
minimal warm (취향저격 그녀 X 찬열 (CHANYEOL)) -- 찬열 (CHANYEOL) -- done
2U -- 강다니엘 -- done
HAVE A GOODNIGHT (취향저격 그녀 X 셔누 (몬스타엑스), 민혁 (몬스타엑스)) -- 셔누 (몬스타엑스), 민혁 (몬스타엑스) -- done
이별자리 -- 이우 -- done
O Sole Mio -- IZ*ONE (아이즈원) -- done
내사람 (Ivy Mix) -- SG 워너비 -- done
안녕 -- 박혜경 -- done
벌써일년 -- 반하나, MJ (써니사이드) -- done
BAAM -- 모모랜드 (MOMOLAND) -- done
거짓말이라도 해서 널 보고싶어 -- 백지영 -- done
나랑 같이 걸을래 (바른연애 길잡이 X 적재) -- 적재 -- done
LOVE DAY (2021) (바른연애 길잡이 X 양요섭, 정은지) -- 양요섭, 정은지 -- done
내 마음의 사진 -- 송가인 -- done
그 한마디 (바른연애 길잡이 X 이하이) -- 이하이 -- done
LOVE DAY -- 양요섭, 정은지 -- done
모든 밤 너에게 (연애혁명 X 민현 (뉴이스트)) -- 민현 (뉴이스트) -- done
No Woman No Cry -- 스토니 스컹크 -- done
음 (Mmmh) -- 카이 (KAI) -- done
하이힐 -- 브레이브걸스 -- done
사랑이란 멜로는 없어 -- 전상근 -- done
맨발의 청춘 -- 벅 -- done
두바둡 (only u) -- 라붐 (LABOUM) -- done
바다 끝 -- 최백호 -- done
집에만 있었지 (바니와 오빠들 X 백아연) -- 백아연 -- done
바람고개 -- 진해성 -- done
Dream Night -- ASTRO (아스트로) -- done
흑백사진 (2021) (바른연애 길잡이 X KCM) -- K

Mazeltov -- 제국의아이들 (ZE:A) -- done
Darling -- 걸스데이 -- done
사고쳤어요 -- 다비치 -- done
질투가 좋아 -- 멜로망스 -- done
떠날꺼야 -- 쎄쎄쎄 -- done
마네킹 -- 이루 -- done
사랑했다 미워했다 (I Loved You) -- 황치열 -- done
썸데이 -- The One (더원) -- done
BIRD -- 하성운 -- done
가끔 이러다 -- 펀치 (Punch) -- done
빨간 운동화 -- 박혜경 -- done
FOR YOU -- 플라워 -- done
No Make Up -- Zion.T -- done
사랑 범벅 (Feat. 챈슬러 of the channels) -- MC몽 -- done
눈물이 주르륵 -- 손담비 -- done
I Don't Care -- 2NE1 -- done
나만 바라봐 -- 태양 -- done
1분 1초 -- 지연 -- done
질투나 -- 거미, 에일리(AILEE) -- done
미쳤어 (Feat. 에릭) -- 손담비 -- done
My Way -- 이수 (엠씨더맥스) -- done
Beverly 1lls -- 수퍼비, myunDo -- done
노래 불러줘요 (Feat. 아이유) -- god -- done
Dry Flower -- SURL (설) -- done
사랑은 다 이러니 -- 홍진영 -- done
TONIGHT -- BIGBANG -- done
들었다 놨다 -- 데이브레이크 (DAYBREAK) -- done
영원한 사랑 -- 핑클 (Fin.K.L) -- done
아미가르 레스토랑 -- 쎄쎄쎄 -- done
나를 잊지 말아요 -- 허각 -- done
빗물이 내려서 -- 김태우 -- done
꿈처럼 내린 -- 다비치 -- done
Endless -- 플라워 -- done
서울 -- 쏜애플 (THORNAPPLE) -- done
슬픈다짐 -- 야다 (Yada) -- done
Mr. Chu -- Apink (에이핑크) -- done
나를 슬프게 하는 사람

원더우먼 -- 씨야, 다비치, 티아라 -- done
천하무적 (Feat. 박장근 Of Supersta) -- MC몽 -- done
너에게 쓰는 편지 Part 2 (Feat. LISA) -- MC몽 -- done
하늘, 바다, 나무, 별의 이야기 -- 조관우 -- done
알아 (Feat. HYNN(박혜원)) -- MC몽 -- done
Luv D.N.A (Feat. 나비) -- MC몽 -- done
Lonely -- 2NE1 -- done
입영열차안에서 -- 김민우 -- done
봄비 (With 알리) -- 김장훈 -- done
물망초 (Feat. 신용재) -- MC몽 -- done
행복해 (Feat. 키겐 of 팬텀) -- 애즈원 -- done
좋은사람 만나요 (Feat. 환희_Fly To The Sky) -- 메이비 (Maybee) -- done
너의 가슴으로 가는 길 -- 먼데이 키즈 (Monday Kiz) -- done
분홍립스틱 -- 송윤아 -- done
오랜만이야 -- 임창정 -- done
꽃 (Feat. Justhis) (Flowers) -- 매드클라운 -- done
호랑나비 (Feat. Gill, 리듬파워, 김흥국) -- 보이비 -- done
어젠 (DUET.환희) -- 송지은 -- done
MY STAR -- 이하이 -- done
마지막 약속 -- 김정민 -- done
미친거니 -- 바이브 -- done
365일 -- 알리 (ALi) -- done
빨리 전화해 (Feat. KUSH) -- MOBB (MINO & BOBBY) -- done
겁나 (Feat. 매드클라운) -- 멜로디데이 (MelodyDay) -- done
모르시나요 -- 다비치 -- done
엄마 -- 라디 (Ra. D) -- done
그게 좋은거야 (Time with you) -- 규현 (KYUHYUN) -- done
전쟁이야 -- 엠블랙 -- done
돌아보지마 (Follow Me & You) -- 이진혁 -- done
아름다운 밤이야 -- 비스트 -- done
나만 알고 싶다 

알아요 -- 양파, 이보람, 소연 -- done
난 좋아 -- 정준일 -- done
키다리 아저씨 -- 백아연 -- done
오랜만이죠 (Feat. 신지수) -- 라디 (Ra. D) -- done
Tonight -- 권순관 -- done
마카롱 -- 걸스데이 -- done
삼각형 -- 러블리즈 -- done
정상을 향한 독주 2 (Feat. 김범수) -- 주석 -- done
사랑해요 -- 김범수 -- done
아무도 모르게 (Prod. by MC몽) -- 치타 (CHEETAH), 에일리(AILEE) -- done
약속 -- 제로 -- done
너, 한눈 팔지마! -- 걸스데이 -- done
꿈을 꾸다 -- 김태우 -- done
집에 가지마 -- GD&TOP -- done
돌아버리겠네 -- 아웃렛 (OUTLET) -- done
About Love -- Red Velvet (레드벨벳) -- done
바다를 나는 거북이 (PROD. 이단옆차기) -- 정세운 -- done
아름다운 그대 (Feat. CHOIZA) -- Crush -- done
전국민 러브송 (Feat. MC몽, 유리 Of Cool) -- 박장근 -- done
TIAMO -- 티아라 -- done
옆에도 눈이 달렸어 (Side, Eye, Moon) -- 윤지성 -- done
가슴앓이 -- 바비 킴 -- done
인연 -- 거미 -- done
Beautiful Target -- B1A4 -- done
힙합 뮤직 (Feat. 임정희) -- 주석 -- done
사랑은 이제 그만 -- 버블 시스터즈 -- done
ANYMORE -- 서인영 -- done
먼저 말해 -- Lim Kim -- done
넌 또 다른 나 -- 이승철 -- done
Hello Bubble -- 걸스데이 -- done
영원한 사랑 -- 씨야, 다비치, 티아라 -- done
Paradise -- 효린 -- done
너의 곁에 -- 변진섭 -- done
어떤 설레임 -- 라디 (Ra. D) -- done
View -- 적재 -- done
G

In [24]:
# 마지막 페이지는 6개라서 따로 크롤링
driver.get(url+str(48))
for i in range(1,7):
        # 음원 1개 누르는 버튼
        music_btn = driver.find_element_by_xpath('//*[@id="market_list"]/div[2]/a['+str(i)+']')
        music_btn.send_keys(Keys.ENTER)
        time.sleep(1)
        
        # 곡정보 보여주는 버튼
        show_info_btn = driver.find_element_by_xpath('//*[@id="market_list"]/div[2]/div/div[5]/div[1]/a')
        show_info_btn.send_keys(Keys.ENTER)
        time.sleep(1)
        
        # 곡제목 크롤링후 리스트에 추가
        title = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[1]/div[2]/p/strong').text
        title_list.append(title)    
        
        # 아티스트명 크롤링후 리스트에 추가
        artist = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[1]/div[2]/em').text
        artist_list.append(artist)
        
        # 연도별 저작권료 크롤링
        five_year_btn =driver.find_element_by_xpath('//*[@id="song_info_royalty"]/div[2]/div/div/div[1]/div/div[1]/div/a[2]')
        five_year_btn.send_keys(Keys.ENTER)
        time.sleep(1)
        
        # 연도별 저작권료 각 연도에 맞게 리스트에 추가
        for year in range(2,6) : 
            fee = driver.find_element_by_xpath('//*[@id="graph1"]/span['+str(year)+']').get_attribute("data-bar-value")
            if year == 2 :
                저작권료_2017.append(fee)
            elif year == 3:
                저작권료_2018.append(fee)
            elif year == 4 : 
                저작권료_2019.append(fee)
            elif year == 5:
                저작권료_2020.append(fee)
        저작권료_2021 = driver.find_element_by_xpath('//*[@id="song_info_royalty"]/div[2]/div/div/div[2]/div/div[1]/strong').text
        최근1년저작권료_1주기준.append(저작권료_2021)
        
        # 매체별 저작권료 분포 크롤링후 각 매체에 맞게 리스트에 추가
        for idx in range(1,7):
            media = driver.find_element_by_xpath('//*[@id="song_info_royalty"]/div[2]/div/div/div[2]/div/div[2]/div[2]/dl['+str(idx)+']/dd').text
            if idx == 1:
                방송.append(media)
            if idx == 2:
                전송.append(media)
            if idx == 3:
                복제.append(media)
            if idx == 4:
                공연.append(media)
            if idx == 5:
                해외.append(media)
            if idx == 6:
                기타.append(media)
        
        # 저작권 공표일자 크롤링후 리스트에 추가
        date = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[2]/section[5]/div[2]/div/dl/dd[1]').text
        저작권공표일자.append(date)
        
        # 작곡가 크롤링후 리스트에 추가
        composition = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[2]/section[5]/div[2]/div/dl/dd[3]').text
        작곡.append(composition)
        
        # 작사가 크롤링후 리스트에 추가
        writer = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[2]/section[5]/div[2]/div/dl/dd[4]').text
        작사.append(writer)
        
        # 편곡자 크롤링후 리스트에 추가
        arrangement = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[2]/section[5]/div[2]/div/dl/dd[5]').text
        편곡.append(arrangement)
        
        
        # 저작권료 지분 각페이지마다 xpath가 달라서 for문으로 조건줘서 크롤링후 리스트에 추가
        for num in range(8,11):
            
            지분 = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[2]/section[5]/div[2]/div/dl/dt['+str(num)+']').text
            if 지분 == '저작권료 지분':
                stake = driver.find_element_by_xpath('//*[@id="page_market"]/div[2]/div[2]/section[5]/div[2]/div/dl/dd['+str(num)+']/p').text
                저작권료지분.append(stake)
                break # 없을시 오류발생

        
        time.sleep(1)
        
        # 디버깅 문장 출력
        print(title,"--",artist,"-- done" ) 
   
        # back_btn - 웹페이지 크기가 작을때만 활성화되는 버튼이라 사이즈 작게하고 크롤링해야함! 
        back_btn = driver.find_element_by_xpath('//*[@id="page_market"]/div[1]/div/a') 
        back_btn.send_keys(Keys.ENTER)

동화 같은 사랑 -- Apink (에이핑크) -- done
Never Ever -- GOT7 (갓세븐) -- done
세상은 요지경 -- 신신애 -- done
슬픈 사랑 -- 기호 -- done
각 (Angle) -- 황치열 -- done
담아 갈게 -- 이무진 -- done


In [25]:
copyright_fee_frm = pd.DataFrame({
    
    '곡명' : title_list,
    '아티스트명': artist_list,
    '저작권료_2017년': 저작권료_2017,
    '저작권료_2018년': 저작권료_2018,
    '저작권료_2019년': 저작권료_2019,
    '저작권료_2020년': 저작권료_2020,
    '최근1년저작권료_1주기준': 최근1년저작권료_1주기준,
    '방송(유무선방송, 라디오 등)': 방송,
    '전송(음원유통사, 유튜브 등)': 전송,
    '복제(음반, 영화, 광고 등)': 복제,
    '공연(노래방, 유원시설 등)': 공연,
    '해외': 해외,
    '기타': 기타,
    '저작권공표일자' : 저작권공표일자,
    '작곡': 작곡,
    '작사': 작사,
    '편곡': 편곡,
    '저작권료 지분':저작권료지분
})
copyright_fee_frm.head()

,곡명,아티스트명,저작권료_2017년,저작권료_2018년,저작권료_2019년,저작권료_2020년,최근1년저작권료_1주기준,"방송(유무선방송, 라디오 등)","전송(음원유통사, 유튜브 등)","복제(음반, 영화, 광고 등)","공연(노래방, 유원시설 등)",해외,기타,저작권공표일자,작곡,작사,편곡,저작권료 지분
0,롤린 (Rollin'),브레이브걸스,"1,340","1,651","3,311","3,490","27,044원","1,151원 (4.3%)","22,741원 (84.1%)","1,013원 (3.7%)","2,136원 (7.9%)",3원 (0%),0원 (0%),2017-03-07,"투챔프 1, 차쿤, 투챔프 2, 용감한형제","차쿤, 용감한형제","투챔프 1, 레드쿠키, 투챔프 2, 용감한형제","1/4,782"
1,축가,전우성 (노을),,"2,182","5,678","7,348","17,608원",0원 (0%),"9,539원 (54.2%)",326원 (1.9%),"7,743원 (43.9%)",0원 (0%),0원 (0%),2018-06-08,"BLACK EDITION (1), 오성훈","이단옆차기2, 최성일, 민연재",박가영,"1/2,000"
2,오늘 그녀가,루그,"14,671","13,704","11,693","14,326","14,626원",0원 (0%),"14,609원 (99.9%)",0원 (0%),0원 (0%),17원 (0.1%),0원 (0%),2008-04-16,"김세진, 김도훈","김세진, 김도훈","김세진, 김도훈",1/300
3,minimal warm (취향저격 그녀 X 찬열 (CHANYEOL)),찬열 (CHANYEOL),,,,,"13,263원",0원 (0%),"13,263원 (100%)",0원 (0%),0원 (0%),0원 (0%),0원 (0%),2020-11-01,헤르쯔 아날로그(HERZ ANALOG),헤르쯔 아날로그(HERZ ANALOG),정동환,"1/3,400"
4,2U,강다니엘,,,,"11,577","11,107원",240원 (2.2%),"10,504원 (94.6%)",117원 (1.1%),73원 (0.7%),173원 (1.4%),0원 (0%),2020-03-24,챈슬러,"챈슬러, 진호현",원작자 사후 70년,"1/5,153"


In [26]:
# csv 파일로 저장하기
copyright_fee_frm.to_csv('뮤직카우_저작권정보.csv',index=False, encoding="utf-8-sig")

In [36]:
copyright_fee_frm['곡명'].value_counts()

Something                                3
Rap Star                                 2
다시는 사랑하지 않고, 이별에 아파하기 싫어                 2
Perhaps Love (사랑인가요) (Prod.By 박근태)       2
창피해                                      2
                                        ..
행복한 나를                                   1
Bungee                                   1
행복해                                      1
화 (Feat. 진실 Of Mad Soul Child) (Fire)    1
혼자만 하는 사랑                                1
Name: 곡명, Length: 919, dtype: int64